In [1]:
# import data science libraries
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

from sklearn.preprocessing import PowerTransformer

import wrangle as wr
import preprocessing_permits as pr
import explore as ex
import model as mo

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pr.get_permits_model_df()
print(f"""Our modeling DataFrame contains {df.shape[0]:,} observations & {df.shape[1]} features""")
df

Our modeling DataFrame contains 8,269 observations & 6 features


,city,state,year,total_high_density_bldgs,total_high_density_units,total_high_density_value
0,Abilene,TX,1997,0.0,0.0,0.0
1,Abilene,TX,1998,0.0,0.0,0.0
2,Abilene,TX,1999,0.0,0.0,0.0
3,Abilene,TX,2000,15.0,192.0,10200000.0
4,Abilene,TX,2001,13.0,192.0,6333000.0
...,...,...,...,...,...,...
8264,Yuma,AZ,2015,0.0,0.0,0.0
8265,Yuma,AZ,2016,0.0,0.0,0.0
8266,Yuma,AZ,2017,8.0,68.0,8986000.0
8267,Yuma,AZ,2018,0.0,0.0,0.0


In [3]:
df["year_diff"] = df.year.diff()

In [4]:
df[(df.year_diff == 1) | (df.year_diff.isna())]

,city,state,year,total_high_density_bldgs,total_high_density_units,total_high_density_value,year_diff
0,Abilene,TX,1997,0.0,0.0,0.0,NaN
1,Abilene,TX,1998,0.0,0.0,0.0,1.0
2,Abilene,TX,1999,0.0,0.0,0.0,1.0
3,Abilene,TX,2000,15.0,192.0,10200000.0,1.0
4,Abilene,TX,2001,13.0,192.0,6333000.0,1.0
...,...,...,...,...,...,...,...
8264,Yuma,AZ,2015,0.0,0.0,0.0,1.0
8265,Yuma,AZ,2016,0.0,0.0,0.0,1.0
8266,Yuma,AZ,2017,8.0,68.0,8986000.0,1.0
8267,Yuma,AZ,2018,0.0,0.0,0.0,1.0


In [5]:
df[(df.city == "Abilene") & (df.state == "TX")]

,city,state,year,total_high_density_bldgs,total_high_density_units,total_high_density_value,year_diff
0,Abilene,TX,1997,0.0,0.0,0.0,NaN
1,Abilene,TX,1998,0.0,0.0,0.0,1.0
2,Abilene,TX,1999,0.0,0.0,0.0,1.0
3,Abilene,TX,2000,15.0,192.0,10200000.0,1.0
4,Abilene,TX,2001,13.0,192.0,6333000.0,1.0
5,Abilene,TX,2002,0.0,0.0,0.0,1.0
6,Abilene,TX,2003,0.0,0.0,0.0,1.0
7,Abilene,TX,2004,0.0,0.0,0.0,1.0
8,Abilene,TX,2005,1.0,14.0,1200000.0,1.0
9,Abilene,TX,2006,1.0,16.0,1500000.0,1.0


In [6]:
df.sample()

,city,state,year,total_high_density_bldgs,total_high_density_units,total_high_density_value,year_diff
3411,Huntington,WV,2001,3.0,151.0,7469000.0,1.0


In [7]:
df.groupby(["city", "state"]).year.count() == 23

city        state
Abilene     TX        True
Akron       OH        True
Albany      GA        True
            NY        True
            OR       False
                     ...  
Yolo        CA       False
York        PA        True
Youngstown  OH        True
Yuba City   CA        True
Yuma        AZ        True
Name: year, Length: 446, dtype: bool

In [8]:
df[(df.city == "Albany") & (df.state == "OR")]

,city,state,year,total_high_density_bldgs,total_high_density_units,total_high_density_value,year_diff
92,Albany,OR,2014,11.0,121.0,11767000.0,-5.0
93,Albany,OR,2015,3.0,15.0,1256000.0,1.0
94,Albany,OR,2016,2.0,10.0,837000.0,1.0
95,Albany,OR,2017,6.0,58.0,5603000.0,1.0
96,Albany,OR,2018,9.0,92.0,9710000.0,1.0
97,Albany,OR,2019,26.0,250.0,23550000.0,1.0


In [9]:
df[(df.city == "Yolo") & (df.state == "CA")]

,city,state,year,total_high_density_bldgs,total_high_density_units,total_high_density_value,year_diff
8171,Yolo,CA,1997,0.0,0.0,0.0,-22.0
8172,Yolo,CA,1998,38.0,608.0,53550000.0,1.0
8173,Yolo,CA,1999,73.0,737.0,49816000.0,1.0
8174,Yolo,CA,2000,16.0,157.0,13396000.0,1.0
8175,Yolo,CA,2001,7.0,38.0,3310000.0,1.0
8176,Yolo,CA,2002,10.0,95.0,8752000.0,1.0


In [10]:
df["year_diff"] = np.where(df.year != 1997, df.year.diff(), 0)

In [11]:
df[(df.city == "Albany") & (df.state == "OR")]

,city,state,year,total_high_density_bldgs,total_high_density_units,total_high_density_value,year_diff
92,Albany,OR,2014,11.0,121.0,11767000.0,-5.0
93,Albany,OR,2015,3.0,15.0,1256000.0,1.0
94,Albany,OR,2016,2.0,10.0,837000.0,1.0
95,Albany,OR,2017,6.0,58.0,5603000.0,1.0
96,Albany,OR,2018,9.0,92.0,9710000.0,1.0
97,Albany,OR,2019,26.0,250.0,23550000.0,1.0


In [12]:
df[(df.city == "Yolo") & (df.state == "CA")]

,city,state,year,total_high_density_bldgs,total_high_density_units,total_high_density_value,year_diff
8171,Yolo,CA,1997,0.0,0.0,0.0,0.0
8172,Yolo,CA,1998,38.0,608.0,53550000.0,1.0
8173,Yolo,CA,1999,73.0,737.0,49816000.0,1.0
8174,Yolo,CA,2000,16.0,157.0,13396000.0,1.0
8175,Yolo,CA,2001,7.0,38.0,3310000.0,1.0
8176,Yolo,CA,2002,10.0,95.0,8752000.0,1.0


In [13]:
df["year_diff"] = np.where(df.year == 1997, 1,df.year.groupby([df.city, df.state]).diff())

In [14]:
df

,city,state,year,total_high_density_bldgs,total_high_density_units,total_high_density_value,year_diff
0,Abilene,TX,1997,0.0,0.0,0.0,1.0
1,Abilene,TX,1998,0.0,0.0,0.0,1.0
2,Abilene,TX,1999,0.0,0.0,0.0,1.0
3,Abilene,TX,2000,15.0,192.0,10200000.0,1.0
4,Abilene,TX,2001,13.0,192.0,6333000.0,1.0
...,...,...,...,...,...,...,...
8264,Yuma,AZ,2015,0.0,0.0,0.0,1.0
8265,Yuma,AZ,2016,0.0,0.0,0.0,1.0
8266,Yuma,AZ,2017,8.0,68.0,8986000.0,1.0
8267,Yuma,AZ,2018,0.0,0.0,0.0,1.0


In [15]:
consecutive = pd.DataFrame(df.groupby(["city", "state"]).year_diff.sum() == 22).reset_index()

In [16]:
consecutive[consecutive.year_diff]

,city,state,year_diff


In [17]:
df

,city,state,year,total_high_density_bldgs,total_high_density_units,total_high_density_value,year_diff
0,Abilene,TX,1997,0.0,0.0,0.0,1.0
1,Abilene,TX,1998,0.0,0.0,0.0,1.0
2,Abilene,TX,1999,0.0,0.0,0.0,1.0
3,Abilene,TX,2000,15.0,192.0,10200000.0,1.0
4,Abilene,TX,2001,13.0,192.0,6333000.0,1.0
...,...,...,...,...,...,...,...
8264,Yuma,AZ,2015,0.0,0.0,0.0,1.0
8265,Yuma,AZ,2016,0.0,0.0,0.0,1.0
8266,Yuma,AZ,2017,8.0,68.0,8986000.0,1.0
8267,Yuma,AZ,2018,0.0,0.0,0.0,1.0


In [18]:
activity = df[df.total_high_density_bldgs > 0]

In [19]:
activity

,city,state,year,total_high_density_bldgs,total_high_density_units,total_high_density_value,year_diff
3,Abilene,TX,2000,15.0,192.0,10200000.0,1.0
4,Abilene,TX,2001,13.0,192.0,6333000.0,1.0
8,Abilene,TX,2005,1.0,14.0,1200000.0,1.0
9,Abilene,TX,2006,1.0,16.0,1500000.0,1.0
13,Abilene,TX,2010,7.0,116.0,3794000.0,1.0
...,...,...,...,...,...,...,...
8256,Yuma,AZ,2007,4.0,69.0,3551000.0,1.0
8257,Yuma,AZ,2008,5.0,33.0,1987000.0,1.0
8258,Yuma,AZ,2009,8.0,48.0,3925000.0,1.0
8261,Yuma,AZ,2012,4.0,64.0,7260000.0,1.0


In [20]:
active_consecutive = pd.DataFrame(activity.groupby(["city", "state"]).year_diff.sum() == 23).reset_index()

In [21]:
active_consecutive

,city,state,year_diff
0,Abilene,TX,False
1,Akron,OH,False
2,Albany,GA,False
3,Albany,NY,True
4,Albany,OR,False
...,...,...,...
438,Yolo,CA,False
439,York,PA,True
440,Youngstown,OH,False
441,Yuba City,CA,False


In [22]:
act_con = active_consecutive[active_consecutive.year_diff]

In [23]:
df = pr.permits_preprocessing_mother_function()
print(f"""Our modeling DataFrame contains {df.shape[0]:,} observations & {df.shape[1]} features""")
df

Our modeling DataFrame contains 2,990 observations & 16 features


,city,state,year,total_high_density_bldgs,total_high_density_units,total_high_density_value,avg_units_per_bldg,value_per_bldg,value_per_unit,city_state_high_density_bldgs_delta_pct,city_state_high_density_units_delta_pct,city_state_high_density_value_delta_pct,market_volume,market_volume_delta_pct,ei,city_state
0,Albany,NY,1997,30.0,425.0,17871000.0,14.166667,5.957000e+05,42049.411765,NaN,NaN,NaN,2.054924e+10,NaN,NaN,Albany_NY
1,Albany,NY,1998,47.0,1038.0,54232000.0,22.085106,1.153872e+06,52246.628131,0.566667,1.442353,2.034637,2.529787e+10,0.231085,2.465010,Albany_NY
2,Albany,NY,1999,39.0,515.0,24484000.0,13.205128,6.277949e+05,47541.747573,-0.170213,-0.503854,-0.548532,2.609590e+10,0.031545,0.437662,Albany_NY
3,Albany,NY,2000,25.0,346.0,16130000.0,13.840000,6.452000e+05,46618.497110,-0.358974,-0.328155,-0.341202,2.742204e+10,0.050818,0.626938,Albany_NY
4,Albany,NY,2001,56.0,502.0,24536000.0,8.964286,4.381429e+05,48876.494024,1.240000,0.450867,0.521141,2.913103e+10,0.062322,1.431902,Albany_NY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2985,York,PA,2015,3.0,53.0,3185000.0,17.666667,1.061667e+06,60094.339623,-0.666667,-0.320513,-0.449724,5.200240e+10,0.313639,0.418895,York_PA
2986,York,PA,2016,3.0,34.0,1566000.0,11.333333,5.220000e+05,46058.823529,0.000000,-0.358491,-0.508320,4.928300e+10,-0.052294,0.518810,York_PA
2987,York,PA,2017,15.0,83.0,10204000.0,5.533333,6.802667e+05,122939.759036,4.000000,1.441176,5.515964,5.158824e+10,0.046775,6.224796,York_PA
2988,York,PA,2018,2.0,26.0,2160000.0,13.000000,1.080000e+06,83076.923077,-0.866667,-0.686747,-0.788318,5.336251e+10,0.034393,0.204643,York_PA


In [24]:
act_con.reset_index(inplace=True)

In [25]:
proj_df = pd.DataFrame(df.groupby(["city", "state"]).year.count() == 23).reset_index()

In [26]:
proj_df

,city,state,year
0,Albany,NY,True
1,Albuquerque,NM,True
2,Allentown,PA,True
3,Anchorage,AK,True
4,Appleton,WI,True
...,...,...,...
125,Tuscaloosa,AL,True
126,Washington,DC,True
127,Wilmington,NC,True
128,Worcester,MA,True


In [27]:
act_con.drop(columns="index", inplace=True)

In [28]:
act_con.rename(columns={"year_diff": "year"}, inplace=True)

In [29]:
(act_con == proj_df).sum()

city     130
state    130
year     130
dtype: int64